In [ ]:
import cirq
import numpy as np
import random as rand
import matplotlib.pyplot as plt
from ising_circuit import IsingCircuit
import json

from cirq import Circuit, ops, linalg, protocols

# Quantum Compression


## Introduction
Goal: Simulate an Ising chain of $n$-qubits, using only $\log n$ qubits. Specifically, the paper does a four qubit Ising chain using two qubits. We must decompose the circuits for compressed simulation into the available gate set. We must also run the experiment often enough so that statistical errors are reduced. Systematic errors must be estimated using independent controlled circuits of similar complexity to the one of interest. Since we are running in simulation via Cirq, this may be less of an issue.

We use the following set up in Cirq for the rest of the tutorial:

In [ ]:
qubits = cirq.LineQubit.range(3)
qubit0, qubit1, qubit2 = qubits

circuit = cirq.Circuit()

## Step 1
Prepare the input state $\rho_{i n} = \frac{1}{2^{m-1}} I^{\otimes m-1} \otimes |+_{y} \rangle \langle+_{y} |$ where $Y |+_{y} \rangle=|+_{y} \rangle$ by applying $S^{\dagger}H$ on qubit $0$, $H$ on an auxilirary qubit $2$, and $CNOT$ on qubits $1$ and $2$ with qubit $1$ being the target and qubit $2$ the control.

In [ ]:
circuit.append([cirq.H(qubit0)])
circuit.append([cirq.S(qubit0)**-1])
circuit.append([cirq.H(qubit2)])
circuit.append([cirq.CNOT(control=qubit2, target=qubit1)])

circuit

## Step 2

Evolve the system up to the desired value of $J$, which in this case is $J = 1$, by decomposing the evolutionary operator $W(J)=\prod_{l=1}^{L(J)} U_{d} R_{l}^{T} R_{0}^{T}$ into the Clifford+T gate set. This is done, rather than decomposing each step in the adiabatic evolution $U_{d} R_{l}^{T} R_{0}^{T}$, in order to keep the total circuit depth practically feasible.

In [ ]:
circuit.append([cirq.H(qubit0)])
circuit.append([cirq.CNOT(control=qubit0, target=qubit1)])
circuit.append([cirq.Z(qubit0)])
circuit.append([cirq.H(qubit0)])
circuit.append([cirq.CNOT(control=qubit0, target=qubit1)])
circuit.append([cirq.S(qubit0)])
circuit.append([cirq.T(qubit1)])
circuit.append([cirq.H(qubit0)])
circuit.append([cirq.Z(qubit1)])
circuit.append([cirq.CNOT(control=qubit0, target=qubit1)])
circuit.append([cirq.T(qubit0)])
circuit.append([cirq.H(qubit0)])
circuit.append([cirq.T(qubit0)])

circuit

Apply $(THS)^4$ on qubit $0$:

In [ ]:
for _ in range(4):
    circuit.append([cirq.S(qubit0)])
    circuit.append([cirq.H(qubit0)])
    circuit.append([cirq.T(qubit0)])

circuit

In [ ]:
circuit.append([cirq.H(qubit0)])
circuit.append([cirq.T(qubit0)])
circuit.append([cirq.H(qubit0)])
circuit.append([cirq.T(qubit0)**-1])
circuit.append([cirq.H(qubit0)])
circuit.append([cirq.T(qubit0)**-1])
circuit.append([cirq.H(qubit0)])
circuit.append([cirq.T(qubit0)])
circuit.append([cirq.Z(qubit0)])
circuit.append([cirq.H(qubit0)])

circuit

## Step 3

Measure $Y$ on qubit $m$ to obtain the magnetization. In this case, $m = \log_{2}(4) - 1 = 1$. By nature of the rotation matrices in the matchgate circuit construction (Jozsa et al.), the $Z$ expectation of qubit $0$ after the circuit has completed $\langle Z_{0} \rangle$ is equal to the $Y$ expectation of qubit $m$ $\langle Y_{m} \rangle$, so a standard basis measurement made. Measuring the state $|0\rangle$ will correspond to an eigenvalue of $+1$ and the state $|1\rangle$ to an eigenvalue of $-1$. The expectation is equal to $\langle Y_{m} \rangle = -M(J)$, the magnetization as a function of $J$ (Hebenstreait et al.).

### Wave Function


Preserve the wave function of the system prior to measurement:

In [ ]:
simulator = cirq.Simulator()
wave_function = simulator.simulate(circuit, qubit_order=qubits)

In [ ]:
wave_function

Calculate the magnetization $M(J)$ from the wave function by decomposing the final state into the $+1$ and $-1$ eigenstates, determining the total probability for each, and computing the expectation $\langle Y_{m} \rangle$:

In [ ]:
final_state = wave_function.final_state

p_1_wfn = (sum([np.absolute(i)**2 for i in final_state[0:4]]))
p_n1_wfn = (sum([np.absolute(i)**2 for i in final_state[4:8]]))

Y_wfn = 1 * p_1_wfn + -1 * p_n1_wfn

M_wfn = -Y_wfn
M_wfn

# Measured Outcome

Apply the standard basis measurement operator to qubit $0$:

In [ ]:
circuit.append([cirq.measure(qubit0, key='x')])
circuit

Repeatedly run the circuit for $n=1,000,000$ iterations:

In [ ]:
n = 1000000
results = simulator.run(circuit, repetitions=n)
results

In [ ]:
hist = results.histogram(key='x')

for k in hist:
    v = hist[k]
    hist[k] = v
    
print(hist)

plt.bar(range(len(hist)), hist.values(), align='center')
plt.xticks(range(len(hist)), list(hist.keys()))

plt.show()

Calculate the magnetization $M(J)$ from the measured outcomes by determining the relative frequency of each eigenstate and computing the expectation $\langle Y_{m} \rangle$:

In [ ]:
f_1_meas = hist[0] / n
f_n1_meas = hist[1] / n

Y_meas = 1 * f_1_meas - 1 * f_n1_meas

M_meas = - Y_meas
M_meas

# Final Results
We can now compare our three results: the measured outcomes (finite number of runs), the wave function (limit of infinite runs), and the theoretical prediction given by the Ising model (Hebenstreait et al.):

In [ ]:
M_theor = -0.81

err_wfn = 100 * (M_theor - M_wfn) / M_theor
err_meas = 100 * (M_theor - M_meas) / M_theor

err_wfn, err_meas

Both of our results have about 5% error, which is consistent with the results of Hebenstreait et al. for a four-qubit spin chain simulation.


# Ising Circuit

Here, we run a much deeper, non-decomposed circuit. The benefit here is that rather than a specific circuit, we can produce circuits that simulate $n$ qubit ising chains with $m = log(n) + 1$ qubits accurately at the cost of circuit depth. The following is a circuit that simulates a four-qubit ising chain like above. The parameters for `IsingCircuit()` are: $n$, the number of qubits in the simulation, $J$, the value we want to evolve our hamiltonian to, $J_{max}$, the highest $J$-value we expect to evolve to, $L$, the number of steps in our evolution minus one, and $T$, the total time for evolution.

In [ ]:
simCircuit = IsingCircuit(4, 1, 2, 200, 10)
simCircuit.circuit

In [ ]:
simulator = cirq.Simulator()
simCircuit.apply_measurement()
result_new = simulator.run(simCircuit.circuit, repetitions=1000)
result_new

In [ ]:
hist = result_new.histogram(key='x')

for k in hist:
    v = hist[k]
    hist[k] = v
    
print(hist)

plt.bar(range(len(hist)), hist.values(), align='center')
plt.xticks(range(len(hist)), list(hist.keys()))

plt.show()

In [ ]:
f_1_meas = hist[0] / (hist[0] + hist[1])
f_n1_meas = hist[1] / (hist[0] + hist[1])

Y_meas = 1 * f_1_meas - 1 * f_n1_meas

M_meas = - Y_meas
M_meas

Again, we can compare the result of the circuit to the result theoretical result of the adiabatic evolution.

In [ ]:
M_theor = -0.81
err_meas = 100 * (M_theor - M_meas) / M_theor
err_meas

Here, we can see that with the larger, non-decomposed circuit, the results are more accurate at the cost of longer runtime.

# Calculate M(J) for J=0 to J=2

To make a comparison between the paper on "Compressed quantum computation using the IBM Quantum Experience", we will recreate the same graph for  $M(J)$ vs $J$ for $J=0$ to $J=2$.

In [ ]:
j_vals = list(np.linspace(0, 2, 13))
j_vals

In [ ]:
def run_circuit(IsingCircuits):
    return [simulator.run(ic.circuit, repetitions=1000) for ic in IsingCircuits]

def get_measurement_outcomes(j_vals, results):
    outcomes = dict()
    for i, r in enumerate(results):
        j = j_vals[i]
        hist = r.histogram(key='x')
        for k in hist:
            v = hist[k]
            hist[k] = v
        
        f_1_meas = hist[0] / (hist[0] + hist[1])
        f_n1_meas = hist[1] / (hist[0] + hist[1])
        Y_meas = 1 * f_1_meas - 1 * f_n1_meas
        M_meas = - Y_meas 
        outcomes[j] = M_meas
    
    return outcomes

In this case, our circuits will simulate 32 qubit ising chains with $L = 200$ and $t = 10$.

In [ ]:
IsingCircuits = [IsingCircuit(32, j, 2, 200, 10) for j in j_vals]

data = [] # a list of the measurement outcomes
# mesaurement outcomes is a dictionary with J -> M(J)
# run the 1000 sims 50 times, take the average
# this takes 20 hours to run on a MBP, try importing the data.json file if interested.
# SKIP THIS BLOCK IF YOU WANT TO AVOID LONG WAIT, AND RUN THE NEXT BLOCK INSTEAD.
num_samples = 50
for _ in range(num_samples):
    results = run_circuit(IsingCircuits)
    measurement_outcomes = get_measurement_outcomes(j_vals, results)
    data.append(measurement_outcomes)
    
# Dump the JSON data
with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
# RUN THIS BLOCK TO USE PAST DATA
with open('data.json', 'r') as f:
    data = json.load(f)
    
data

In [ ]:
# get the average M(J) and std dev
avg_data = dict() # J -> (M(J) stddev, M(J) avg)
for j in list(map(str, j_vals)):
    m_vals = []
    for d in data:
        m = d[j]
        m_vals.append(m)
    stddev = np.std(m_vals)
    mean = np.mean(m_vals)
    avg_data[j] = (stddev, mean)
    
avg_data

In [ ]:
J, M = zip(*avg_data.items())
J = list(map(float, J))
stddev_M, mean_M = list(zip(*M)) 
print("J values:", J)
print("M(J) avg:", mean_M)
print("M(J) stddev:", stddev_M)

In [ ]:
# J, M = zip(*measurement_outcomes.items())
# plt.plot(J, M, 'r+')
# plt.xlim(0, 2)  # decreasing time
# plt.grid(True)
# plt.title('Cirq Quantum Compression')
# plt.xlabel('J')
# plt.ylabel('M(J)')
# plt.xticks(list(np.linspace(0, 2, 5)))
# plt.yticks(list(np.linspace(-1, 0, 6)))

In [ ]:
# 50 runs of 1000 repetitions on circuit, plotted with avg and stddev
plt.errorbar(J, mean_M, yerr=stddev_M, fmt='+', color='r',
             ecolor='black', elinewidth=1, capsize=1);
plt.xlim(-0.05, 2.05)  # decreasing time
plt.grid(True)
plt.title('Cirq Quantum Compression')
plt.xlabel('J')
plt.ylabel('M(J)')
plt.xticks(list(np.linspace(0, 2, 5)))
plt.yticks(list(np.linspace(-1, 0, 6)))
plt.savefig('final_result.png', dpi=288)